In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960

In [ ]:
import numpy as np
import tifffile
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, Input, Conv2DTranspose, Concatenate, Activation, Reshape, Lambda
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.data import Dataset
import keras.backend as K
tf.config.optimizer.set_jit(True)

In [ ]:
def conv_block(X_input,output_channels,kernel_size,strides,padding):

    X_input = Conv2D(output_channels, kernel_size = kernel_size, strides = strides, padding = padding, kernel_initializer = tf.keras.initializers.HeNormal())(X_input)

    X_input = BatchNormalization()(X_input)

    X_input = Activation('relu')(X_input)

    return X_input

In [ ]:
def encoder_block(X_input,output_channels,kernel_size,strides):

    X_input = conv_block(X_input,output_channels, kernel_size=kernel_size, strides=strides, padding = 'same')

    X_concat = conv_block(X_input,output_channels, kernel_size=kernel_size, strides=strides, padding = 'same')

    X_down = Conv2D(output_channels, kernel_size = (2,2), strides = 2, padding = 'valid', activation = 'relu')(X_concat)

    return X_concat, X_down







In [ ]:
def middle_block(X_input,output_channels,kernel_size,strides):

    X_input = conv_block(X_input,output_channels, kernel_size=kernel_size, strides=strides, padding = 'same')

    X_input = conv_block(X_input,output_channels, kernel_size=kernel_size, strides=strides, padding = 'same')

    X_upward = Conv2DTranspose(output_channels, kernel_size = (3,3), strides = (2, 2), padding = 'same')(X_input)



    return X_upward

In [ ]:
def decoder_block(X_concat,X_upward,output_channels,kernel_size,strides, last = False):

    X_input = Concatenate()([X_concat,X_upward])

    X_input = Dropout(0.1)(X_input)

    X_input = conv_block(X_input,output_channels, kernel_size,strides,padding='same')

    X_input = conv_block(X_input,output_channels,kernel_size,strides,padding='same')

    if not last:

        X_input = Conv2DTranspose(output_channels, kernel_size = (3,3), strides = (2, 2), padding = 'same')(X_input)

    return X_input



In [ ]:
def dice_loss(y_true, y_pred):

    def dice_coef(y_true, y_pred, smooth=1):

        intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
        return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)


    return 1-dice_coef(y_true, y_pred)


In [ ]:
def Unet():

    ks = (3,3)

    s = 1

    X_input = Input(shape=(256,256,3))

    X_rescale = Rescaling(scale = 1./255)(X_input)

    X_concat1, X_down = encoder_block(X_rescale,output_channels = 32, kernel_size = ks, strides = s)

    X_concat2, X_down = encoder_block(X_down, output_channels = 64, kernel_size = ks, strides = s)

    X_concat3, X_down = encoder_block(X_down, output_channels = 128, kernel_size = ks, strides = s)

    X_upward1 = middle_block(X_down, output_channels = 128, kernel_size = ks, strides = s)

    X_upward2 = decoder_block(X_concat3, X_upward1, output_channels = 64, kernel_size = ks, strides = s)

    X_upward3 = decoder_block(X_concat2, X_upward2, output_channels = 32, kernel_size = ks, strides = s)

    X_last = decoder_block(X_concat1, X_upward3, output_channels = 32, kernel_size = ks, strides = s, last = True)

    X_last = Conv2D(1, kernel_size = ks, strides = s, padding = 'same', activation = 'sigmoid')(X_last)

    X_output = Reshape((256,256))(X_last)

    model = Model(inputs = X_input, outputs= X_output)

    return model





In [ ]:
model = Unet()

In [ ]:
model.compile(loss = dice_loss, optimizer = Adam(learning_rate=0.0003), metrics = ['binary_accuracy'])

In [ ]:
model.summary()

In [ ]:
STDEV_THRESHOLD = 10
overlap = 36

In [ ]:
def boxes(file):

    image = tifffile.imread(file)
    image = np.squeeze(image)
    if image.shape[0] == 3: #channels_first
        image = np.transpose(image, (1, 2, 0))
    assert image.shape[2] != 4
    len_image, wid_image, _ = image.shape
    list_range_len_image = list(range(0, len_image, 256-overlap))
    random.shuffle(list_range_len_image)
    list_range_wid_image = list(range(0, wid_image, 256-overlap))
    for i in list_range_len_image:
        random.shuffle(list_range_wid_image)
        for j in list_range_wid_image:
            output_array = np.array(image[i:i+256,j:j+256])
            if output_array.shape == (256, 256, 3):
                yield ((i, j), output_array)

In [ ]:
INPUT_DIR = "../input/hubmap-kidney-segmentation/train"
LABEL_DIR = "../input/hubmaplabeling"
file_ids = [x.replace("-label.npy", "") for x in os.listdir("../input/hubmaplabeling")]

def iterate_through_training_images():
    random.shuffle(file_ids)
    for file_id in file_ids:
        label_arr = np.load(f"../input/hubmaplabeling/{file_id}-label.npy", mmap_mode="r")
        for (x, y), input_tile in boxes(f"{INPUT_DIR}/{file_id}.tiff"):
            if np.std(input_tile.ravel()) >= STDEV_THRESHOLD:
                input_tile = tf.convert_to_tensor(input_tile, tf.float32)
                label_tile = tf.convert_to_tensor(label_arr[x:x+256, y:y+256], tf.float32)
                yield input_tile, label_tile
        del label_arr

In [ ]:
BATCH_SIZE = 32

In [ ]:
dataset = Dataset.from_generator(iterate_through_training_images,
                                            output_signature=(
                                                 tf.TensorSpec(shape=(256, 256, 3), dtype=tf.float32),
                                                 tf.TensorSpec(shape=(256, 256), dtype=tf.float32)))
dataset = dataset.shuffle(100)

validation_dataset = dataset.take(500).batch(1)

training_dataset = dataset.skip(500).batch(BATCH_SIZE)

In [ ]:
model.fit(training_dataset, epochs = 7, validation_data = validation_dataset)

In [ ]:
model.evaluate(validation_dataset)
model.save('model.h5')